# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [18]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")



def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')
        
    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [9]:
# %load samples.py
version = 'v38b'

#      Particle('ele', PID.electron),
#                  Particle('ele_A', PID.electron, '1.4 < abseta < 1.7'),
#                  Particle('ele_B', PID.electron, '1.7 <= abseta <= 2.8'),
#                  Particle('ele_C', PID.electron, 'abseta > 2.8'),
#                  Particle('photon', PID.photon),
#                  Particle('photon_A', PID.photon, '1.4 < abseta < 1.7'),
#                  Particle('photon_B', PID.photon, '1.7 <= abseta <= 2.8'),
#                  Particle('photon_C', PID.photon, 'abseta > 2.8'),

particles = ''

sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']




sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_ele = [sample_ele_flat2to100_PU0,sample_ele_flat2to100_PU200]
# samples_ele = [sample_ele_flat2to100_PU200]

samples_photon = [sample_gPt35_PU0,sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0,sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]




tpset_labels = {'DEF' : 'NNDR',
                'DEF_em': 'NNDR + EGID',
                'DEF_pt20': 'NNDR, p_{T}^{L1}>20GeV',
                'DEF_pt20_em': 'NNDR + EGID, p_{T}^{L1}>20GeV',
                'DEF_pt25': 'NNDR, p_{T}^{L1}>25GeV',
                'DEF_pt25_em': 'NNDR + EGID, p_{T}^{L1}>25GeV',
                'DEF_pt30': 'NNDR, p_{T}^{L1}>30GeV',
                'DEF_pt30_em': 'NNDR + EGID, p_{T}^{L1}>30GeV'}


particle_labels = {'ele': 'all #eta',
                   'elePt20': 'p_{T}^{GEN}>20GeV',
                   'elePt30': 'p_{T}^{GEN}>30GeV',
                   'elePt40': 'p_{T}^{GEN}>40GeV',
                   'eleA' : '1.4 < |#eta^{GEN}| < 1.7',
                   'eleB' : '1.7 < |#eta^{GEN}| < 2.8',
                   'eleC' : '|#eta^{GEN}| > 2.8',
                   'eleD' : '|#eta^{GEN}| < 2.4',
                   'all'  : 'all #eta^{GEN}',
                   'etaA': '#eta^{GEN} < 1.7',
                   'etaB': '1.7 < |#eta^{GEN}| < 2.8)',
                   'etaC': '|#eta^{GEN}| > 2.8',
                   'etaD': '|#eta^{GEN}| < 2.4',
                   'etaE': '1.7 < |#eta^{GEN}| < 2.7'}





samples = []


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v38b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v38b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v38b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v38b.root does not exist


In [10]:
samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD']

histo_all_df = pd.DataFrame(columns=['histo', 'name', 'sample_name', 'pu', 'tp_set', 'particle', 'text'])


for sample in samples:
    for tp_set in tp_sets:
        for particle in particle_labels.keys():
            histo_all_df = histo_all_df.append({'name' : 'h_eff',
                                                'histo' : histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff,
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)
            histo_all_df = histo_all_df.append({'name' : 'h_num',
                                                'histo' : histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num,
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)

for sample in samples:
    for tp_set in ['DEF_pt20', 'DEF_pt25', 'DEF_pt30', 'DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em']:
        TP_set_den = 'DEF'
        if 'em' in tp_set:
            TP_set_den = 'DEF_em'
        for particle in particle_labels.keys():
#             print 'TP set: {} den: {}'.format(tp_set, TP_set_den)
            histo_den = histo_all_df[(histo_all_df.name == 'h_num') & 
                                     (histo_all_df.sample_name == sample.name) & 
                                     (histo_all_df.particle == particle) & 
                                     (histo_all_df.tp_set == TP_set_den)].histo
            histo_num = histo_all_df[(histo_all_df.name == 'h_num') & 
                                     (histo_all_df.sample_name == sample.name) & 
                                     (histo_all_df.particle == particle) & 
                                     (histo_all_df.tp_set == tp_set)].histo
            histo_all_df = histo_all_df.append({'name' : 'h_turnon',
                                                'histo' : histos.HistoEff(histo_num.values[0], histo_den.values[0], False),
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)


            
   

In [11]:
histo_all_df

,histo,name,sample_name,pu,tp_set,particle,text
0,<l1THistos.HistoEff instance at 0x11523bb00>,h_eff,ele_flat2to100_PU0,PU0,DEF,eleA,"NNDR, 1.4 < |#eta^{GEN}| < 1.7"
1,<GenParticleHistos h_effNum_DEF_eleA>,h_num,ele_flat2to100_PU0,PU0,DEF,eleA,"NNDR, 1.4 < |#eta^{GEN}| < 1.7"
2,<l1THistos.HistoEff instance at 0x1151eed40>,h_eff,ele_flat2to100_PU0,PU0,DEF,eleB,"NNDR, 1.7 < |#eta^{GEN}| < 2.8"
3,<GenParticleHistos h_effNum_DEF_eleB>,h_num,ele_flat2to100_PU0,PU0,DEF,eleB,"NNDR, 1.7 < |#eta^{GEN}| < 2.8"
4,<l1THistos.HistoEff instance at 0x11523b488>,h_eff,ele_flat2to100_PU0,PU0,DEF,eleC,"NNDR, |#eta^{GEN}| > 2.8"
5,<GenParticleHistos h_effNum_DEF_eleC>,h_num,ele_flat2to100_PU0,PU0,DEF,eleC,"NNDR, |#eta^{GEN}| > 2.8"
6,<l1THistos.HistoEff instance at 0x1151ee680>,h_eff,ele_flat2to100_PU0,PU0,DEF,eleD,"NNDR, |#eta^{GEN}| < 2.4"
7,<GenParticleHistos h_effNum_DEF_eleD>,h_num,ele_flat2to100_PU0,PU0,DEF,eleD,"NNDR, |#eta^{GEN}| < 2.4"
8,<l1THistos.HistoEff instance at 0x1151eed88>,h_eff,ele_flat2to100_PU0,PU0,DEF,elePt20,"NNDR, p_{T}^{GEN}>20GeV"
9,<GenParticleHistos h_effNum_DEF_elePt20>,h_num,ele_flat2to100_PU0,PU0,DEF,elePt20,"NNDR, p_{T}^{GEN}>20GeV"


## Effinciency vs $\eta$

In [7]:
line_m1p7 = ROOT.TLine(-1.7,0, -1.7, 1.1)
line_m1p7.SetLineStyle(2)
line_p1p7 = ROOT.TLine(1.7,0, 1.7, 1.1)
line_p1p7.SetLineStyle(2)
line_m2p8 = ROOT.TLine(-2.8,0, -2.8, 1.1)
line_m2p8.SetLineStyle(2)
line_p2p8 = ROOT.TLine(2.8,0, 2.8, 1.1)
line_p2p8.SetLineStyle(2)

In [12]:

for particle in ['ele', 'elePt20', 'elePt30', 'elePt40']:
    for tp_set in ['DEF', 'DEF_em']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff')]
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_eta, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_m1p7.Draw('same')
        line_p1p7.Draw('same')
        line_m2p8.Draw('same')
        line_p2p8.Draw('same')

## Efficiency vs $p_{T}$

In [16]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [19]:

for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff')]
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
#         print h_eff_vs_pt
        drawGraphsSame(h_eff_vs_pt, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
        
        

## Turn-on curves

In [20]:

for particle in particles:
    for tp_set in ['DEF_pt20', 'DEF_pt25', 'DEF_pt30', 'DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                                  (histo_all_df.particle == particle) &
                                  (histo_all_df.name == 'h_turnon')]
            
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_pt, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
          
        


In [ ]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



In [ ]:
titles_df

In [ ]:
dir(histo_df.ele['DEF'][0])


In [ ]:
histo_df.loc['DEF', 'ele']

In [ ]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [ ]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

In [ ]:
histo_den_df

In [ ]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [ ]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [ ]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
